## Image Panorama Stitching with OpenCV

This is the source code and notes from this [article](https://pyimagesearch.com/2018/12/17/image-stitching-with-opencv-and-python/) it is an introduction to image stitching using the OpenCV library. 

### Notes:
This file uses imutiles package to check the version of OpenCV that the user is using. To install with conda run the following command in your terminal.
`conda install -c conda-forge imutils`

If using pip install by running this command.
`pip install imutils`

In [ ]:
from imutils import paths
from os.path import join
from os import walk
import matplotlib.pyplot as plt
import numpy as np
import imutils
import cv2

In [ ]:
# images used for the stitching process
print("loading images...")
for (dir_root, dir_names, file_names) in walk("../images/scottsdale/"):
    img_paths = [join(dir_root, name) for name in file_names]
    images = [cv2.imread(path) for path in img_paths]

print(f"Done {len(images)} images where loaded.")
for img in images:
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
# initialize OpenCV's image stitcher object and then perform the image stitching
print("stitching images...")
stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
(status, stitched_img) = stitcher.stitch(images)

# if the status is '0', then OpenCV successfully performed image stitching
output_path = "../images/outputs/stitched_image.png"

if status == 0:
    print(f"Done stitching images with status {status}.")
    # write the output stitched image to disk
    cv2.imwrite(output_path, stitched_img)
    # display the output stitched image to our screen
    plt.imshow(cv2.cvtColor(stitched_img, cv2.COLOR_BGR2RGB))
    # as opencv loads in BGR format by default, we want to show it in RGB.
    plt.show()
else:
    print(f"image stitching failed with status ({status})")

### Issues with this implementation 

Notice those black regions surrounding the panorama. With this implementation our output image will have the black regions. The regions are from performing the perspective warps required to construct the panorama. The is done by the OpenCV library behind the scenes.

## A better image stitcher

We will use a hack to crop the black regions from the stitched image.

In [ ]:
output_path = "../images/outputs/better_stitched.png"

if status == 0:
	# create a 10 pixel border surrounding the stitched image
	print("cropping the stitched image...")
	stitched = cv2.copyMakeBorder(stitched_img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, (0, 0, 0))
	# convert the stitched image to grayscale and threshold it
	# such that all pixels greater than zero are set to 255
	# (foreground) while all others remain 0 (background)
	gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
	thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

    # find all external contours in the threshold image then find the *largest* contour 
    # which will be the contour/outline of the stitched image
	cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	c = max(cnts, key=cv2.contourArea)

	# allocate memory for the mask which will contain the
	# rectangular bounding box of the stitched image region
	mask = np.zeros(thresh.shape, dtype="uint8")
	(x, y, w, h) = cv2.boundingRect(c)
	cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

    # create two copies of the mask: one to serve as our actual
	# minimum rectangular region and another to serve as a counter
	# for how many pixels need to be removed to form the minimum
	# rectangular region
	minRect = mask.copy()
	sub = mask.copy()

	# keep looping until there are no non-zero pixels left in the
	# subtracted image
	while cv2.countNonZero(sub) > 0:
		# erode the minimum rectangular mask and then subtract
		# the thresholded image from the minimum rectangular mask
		# so we can count if there are any non-zero pixels left
		minRect = cv2.erode(minRect, None)
		sub = cv2.subtract(minRect, thresh)


	# find contours in the minimum rectangular mask and then
	# extract the bounding box (x, y)-coordinates
	cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	c = max(cnts, key=cv2.contourArea)
	(x, y, w, h) = cv2.boundingRect(c)
	# use the bounding box coordinates to extract the our final
	# stitched image
	stitched_img = stitched_img[y:y + h, x:x + w]

	# write the output stitched image to disk
	cv2.imwrite(output_path, stitched_img)
	
	plt.imshow(cv2.cvtColor(stitched_img, cv2.COLOR_BGR2RGB))
	plt.show()